<a href="https://colab.research.google.com/github/RamziRBM/lab-natural-language-processing/blob/main/your-code/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [17]:
import os, re, string, numpy as np, pandas as pd, matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,5)

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development.

In [5]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_train = pd.read_csv("https://github.com/RamziRBM/lab-natural-language-processing/raw/refs/heads/main/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
#data_train = data_train.head(1000)
print(data_train.shape)
data_train.fillna("",inplace=True)

(5964, 2)


In [9]:
X_raw = data_train["text"]
y = data_train["label"]

X_train_raw, X_valid_raw, y_train, y_valid = train_test_split(
    X_raw, y, test_size=0.20, stratify=y, random_state=42
)

print("Train/valid sizes:", X_train_raw.shape, X_valid_raw.shape)

Train/valid sizes: (4771,) (1193,)


### Let's divide the training and test set into two partitions

In [6]:
# Your code
## Read Data for the Fraudulent Email Kaggle Challenge
data_test = pd.read_csv("https://github.com/RamziRBM/lab-natural-language-processing/raw/refs/heads/main/data/kg_test.csv",encoding='latin-1')

# Reduce the training set to speed up development.
# Modify for final system
#data_test = data_test.head(1000)
print(data_test.shape)
data_test.fillna("",inplace=True)

(5964, 1)


## Data Preprocessing

In [11]:
import nltk
for pkg, path in [("punkt", "tokenizers/punkt"),
                  ("stopwords","corpora/stopwords")]:
    try:
        nltk.data.find(path)
    except LookupError:
        nltk.download(pkg)

STOPWORDS = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [13]:
# Your code
from bs4 import BeautifulSoup, Comment
import re
import pandas as pd


# HTML strip

def strip_html_layers(html):
    """
    1) Remove inline JS/CSS
    2) Remove HTML comments
    3) Remove remaining tags
    Return plain text (no tags).
    """
    if pd.isna(html):
        return ""

    soup = BeautifulSoup(str(html), "html.parser")

    # (1) Remove inline JS/CSS
    #    a) <script> and <style> blocks
    for tag in soup(["script", "style"]):
        tag.decompose()
    #    b) inline event handlers / inline style attributes (onclick, onload, style, etc.)
    for tag in soup(True):
        # collect then delete to avoid runtime mutation issues
        for attr in list(tag.attrs):
            al = attr.lower()
            if al == "style" or al.startswith("on"):  # on* = onclick, onload...
                del tag.attrs[attr]

    # (2) Remove HTML comments (do this before stripping tags in regex approaches; with BS it's explicit)
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        c.extract()

    # (3) Remove remaining tags: get only the text content
    text = soup.get_text(separator=" ", strip=True)

    return text

In [14]:


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Optional: normalize the extracted text (letters only, lowercase, collapse spaces)

for pkg, path in [("punkt","tokenizers/punkt"), ("stopwords","corpora/stopwords")]:
    try:
        nltk.data.find(path)
    except LookupError:
        nltk.download(pkg)

STOPWORDS = set(stopwords.words("english"))

def normalize_text(plain_text: str, remove_stopwords=True):
    if pd.isna(plain_text):
        return ""
    t = plain_text.lower()
    t = re.sub(r"\d+", " ", t)          # remove digits
    t = re.sub(r"[^a-z\s]", " ", t)     # keep letters/spaces
    t = re.sub(r"\s+", " ", t).strip()

    if not remove_stopwords:
        return t

    tokens = [w for w in word_tokenize(t) if w not in STOPWORDS and len(w) > 1]
    return " ".join(tokens)



- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters

- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [15]:
# Your code
import re
import pandas as pd

# 3) Normalize exact
def normalize_exact(text: str) -> str:
    """
    Apply these steps IN ORDER:
    1) Remove all special characters (keep letters/numbers/spaces)
    2) Remove numbers
    3) Remove all single characters
    4) Remove single characters from the start
    5) Substitute multiple spaces with single space
    6) Remove prefixed b (e.g., b'...') if present
    7) Convert to lowercase
    """
    if pd.isna(text):
        return ""
    s = str(text)

    # 1) Remove special characters (keep letters, digits, whitespace)
    s = re.sub(r"[^A-Za-z0-9\s]", " ", s)

    # 2) Remove numbers
    s = re.sub(r"\d+", " ", s)

    # 3) Remove all single characters (isolated a–z/A–Z)
    s = re.sub(r"\b[a-zA-Z]\b", " ", s)

    # 4) Remove single characters from the start (e.g., leading 'a ' at line start)
    s = re.sub(r"^\s*[a-zA-Z]\s+", " ", s)

    # 5) Substitute multiple spaces with single space
    s = re.sub(r"\s+", " ", s).strip()

    # 6) Remove prefixed 'b' (handles byte-literal-like strings such as b'...'/b"...")
    #    If your text literally includes leading b'...' from decoding issues, this strips it.
    m = re.match(r"""^b(?:['"])(.*)(?:['"])$""", s)
    if m:
        s = m.group(1).strip()

    # 7) Convert to lowercase
    s = s.lower()

    return s


In [21]:
from bs4 import MarkupResemblesLocatorWarning
import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [22]:
# 1) HTML → text (structure removal only)
X_train_nohtml = X_train_raw.apply(strip_html_layers)
X_valid_nohtml = X_valid_raw.apply(strip_html_layers)

# 2) Plain-text normalization/token filtering (still per split)
X_train_clean = X_train_nohtml.apply(normalize_text)
X_valid_clean = X_valid_nohtml.apply(normalize_text)

# 3) Normalize exact
X_train_clean = X_train_raw.apply(normalize_exact)
X_valid_clean = X_valid_raw.apply(normalize_exact)

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
# Your code

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [23]:
# Your code
import re
from typing import List
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

# --- Optional lemmatizers (try-import; safe if missing) ---
_use_spacy = False
_use_wordnet = False
try:
    import spacy
    _nlp = spacy.load("en_core_web_sm", disable=["parser","ner","textcat"])
    _use_spacy = True
except Exception:
    try:
        # Try NLTK WordNet if present
        import nltk
        from nltk.stem import WordNetLemmatizer
        from nltk.corpus import wordnet as wn
        lemmatizer = WordNetLemmatizer()
        _use_wordnet = True
    except Exception:
        pass

# --- Tiny rule-based lemmatizer (fallback) ---
_IRREGULAR = {
    "bought":"buy","brought":"bring","went":"go","gone":"go","saw":"see","seen":"see",
    "was":"be","were":"be","am":"be","is":"be","are":"be","been":"be",
    "did":"do","done":"do","has":"have","had":"have",
    "ran":"run","better":"good","best":"good","worse":"bad","worst":"bad",
}
def _rule_based_lemma(tok: str) -> str:
    if tok in _IRREGULAR:
        return _IRREGULAR[tok]
    # Fix common Porter artifacts: happi -> happy, studi -> study, respons -> respons (leave)
    if tok.endswith("i") and len(tok) > 2:
        return tok[:-1] + "y"
    # Plural-ish cleanups
    if tok.endswith("ves") and len(tok) > 3:      # knives -> knive (approx) -> prefer 'f' rule
        return tok[:-3] + "f"
    if tok.endswith("ies") and len(tok) > 3:      # parties -> party
        return tok[:-3] + "y"
    if tok.endswith("s") and len(tok) > 3 and not tok.endswith("ss"):
        return tok[:-1]
    return tok

def _tokenize_letters(s: str) -> List[str]:
    # keep letters only, lowercase
    return re.findall(r"[a-zA-Z]+", s.lower())

def stem_then_lemmatize(text: str) -> str:
    if not isinstance(text, str):
        text = "" if text is None else str(text)

    # 1) tokenize (letters only)
    toks = _tokenize_letters(text)
    if not toks:
        return ""

    # 2) stemming first
    stems = [stemmer.stem(t) for t in toks]

    # 3) then lemmatization
    if _use_spacy:
        # Let spaCy lemmatize the *stemmed* string
        doc = _nlp(" ".join(stems))
        lemmas = [t.lemma_.lower() for t in doc if t.lemma_.strip()]
    elif _use_wordnet:
        # Try both noun/verb, pick the shorter (very simple heuristic)
        def wn_lemma(tok):
            n = lemmatizer.lemmatize(tok, pos="n")
            v = lemmatizer.lemmatize(tok, pos="v")
            return n if len(n) <= len(v) else v
        lemmas = [wn_lemma(t) for t in stems]
    else:
        # Rule-based fallback (no downloads needed)
        lemmas = [_rule_based_lemma(t) for t in stems]

    # 4) collapse whitespace
    return " ".join(lemmas)

In [24]:
X_train_norm = X_train_raw.apply(stem_then_lemmatize)
X_valid_norm = X_valid_raw.apply(stem_then_lemmatize)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [25]:
# Your code
# 1. Build token solumns for ham & spam
# assuming the training dataframe is data_train with columns:
# text;
# label (0 = ham, 1=spam)

import re
from collections import Counter

def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = text.split()
    return tokens

data_train["tokens"] = data_train["text"].apply(tokenize)

# 2. Split ham & spam tokens
ham_tokens = [token for tokens in data_train[data_train["label"] == 0]["tokens"] for token in tokens]
spam_tokens = [token for tokens in data_train[data_train["label"] == 1]["tokens"] for token in tokens]

ham_counts = Counter(ham_tokens)
spam_counts = Counter(spam_tokens)

top_ham = ham_counts.most_common(10)
top_spam = spam_counts.most_common(10)

# 3. Show Top 10 Words in Each Class
print("Top 10 HAM words:")
for word, count in ham_counts.most_common(10):
    print(f"{word:<12} {count}")

print("\nTop 10 SPAM words:")
for word, count in spam_counts.most_common(10):
    print(f"{word:<12} {count}")

# 4. Comparative Analysis
def comparative_analysis(top_ham, top_spam, n=10):
    """
    Compare top Bag-of-Words terms between HAM and SPAM messages.
    Input should be Counter().most_common() lists.
    """
    print("\n=== Comparative Word Analysis (Top {} words) ===".format(n))
    print("{:<15} {:<10} | {:<15} {:<10} | {:<10}".format("HAM Word", "Count", "SPAM Word", "Count", "Dominant"))
    print("-" * 70)

    for i in range(n):
        ham_word, ham_count = top_ham[i] if i < len(top_ham) else ("-", 0)
        spam_word, spam_count = top_spam[i] if i < len(top_spam) else ("-", 0)

        # Determine dominance
        if ham_count > spam_count:
            dominant = f"HAM ({ham_count // (spam_count + 1)}x)"
        elif spam_count > ham_count:
            dominant = f"SPAM ({spam_count // (ham_count + 1)}x)"
        else:
            dominant = "Equal"

        print("{:<15} {:<10} | {:<15} {:<10} | {:<10}".format(
            ham_word, ham_count, spam_word, spam_count, dominant
        ))

comparative_analysis(top_ham, top_spam)

Top 10 HAM words:
the          14282
to           8439
and          6836
of           6542
in           5114
a            5109
s            3574
that         3247
is           2914
for          2796

Top 10 SPAM words:
the          39757
to           31509
of           27934
and          21762
i            20573
in           17976
you          17941
a            16032
this         14583
e            14526

=== Comparative Word Analysis (Top 10 words) ===
HAM Word        Count      | SPAM Word       Count      | Dominant  
----------------------------------------------------------------------
the             14282      | the             39757      | SPAM (2x) 
to              8439       | to              31509      | SPAM (3x) 
and             6836       | of              27934      | SPAM (4x) 
of              6542       | and             21762      | SPAM (3x) 
in              5114       | i               20573      | SPAM (4x) 
a               5109       | in              17976      

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x))

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x))

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [27]:
# Your code
# Your code
from sklearn.feature_extraction.text import CountVectorizer

# assume X_train_raw, X_valid_raw exist (from train_test_split)
vectorizer = CountVectorizer(
    lowercase=True,
    stop_words="english",     # optional
    ngram_range=(1,1)         # (1,1)=unigrams. Can use (1,2) for bigrams
)

# Learn vocabulary on train only
X_train_bow = vectorizer.fit_transform(X_train_raw)

# Transform validation with same vocabulary
X_valid_bow = vectorizer.transform(X_valid_raw)

print("Train matrix shape:", X_train_bow.shape)
print("Valid matrix shape:", X_valid_bow.shape)

import numpy as np

nnz_train = X_train_bow.nnz
total = np.prod(X_train_bow.shape)
print(f"Sparsity: {1 - nnz_train/total:.4f} (fraction of zeros)")

vocab = vectorizer.get_feature_names_out()
print("Sample vocab:", vocab[:20])

Train matrix shape: (4771, 69056)
Valid matrix shape: (1193, 69056)
Sparsity: 0.9988 (fraction of zeros)
Sample vocab: ['00' '000' '0000' '00000' '000000' '00000e2511c8' '00000e2511c8print'
 '0000642ec858344d00000000' '0000647cc858344d00000000' '000066' '000080'
 '0000ff' '00020500' '0008' '000999' '000m' '000million' '000ud' '000us'
 '000usd']


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [28]:
tfidf = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1,2),
    min_df=3,
    max_df=0.90,
    sublinear_tf=True
)


In [31]:
# Split-safe (recommended for modeling)
X_train_tfidf = tfidf.fit_transform(X_train_raw)   # learn vocab on TRAIN only
X_valid_tfidf = tfidf.transform(X_valid_raw)       # reuse same vocab

print("Train TF-IDF shape:", X_train_tfidf.shape)
print("Valid TF-IDF shape:", X_valid_tfidf.shape)
print("Vocab size:", len(tfidf.get_feature_names_out()))

Train TF-IDF shape: (4771, 57912)
Valid TF-IDF shape: (1193, 57912)
Vocab size: 57912


In [32]:
# Vectorizes the entire dataset (for EDA only)
all_text = data_train["text"].fillna("").astype(str)
X_all_tfidf = tfidf.fit_transform(all_text)  # fits on ALL data

print("All-data TF-IDF shape:", X_all_tfidf.shape)
print("Vocab size (all-data):", len(tfidf.get_feature_names_out()))

All-data TF-IDF shape: (5964, 70483)
Vocab size (all-data): 70483


## And the Train a Classifier?

In [33]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1,2),
    min_df=3,
    max_df=0.90,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(X_train_raw)   # fit on TRAIN only
X_valid_tfidf = tfidf.transform(X_valid_raw)
print("Train:", X_train_tfidf.shape, " Valid:", X_valid_tfidf.shape)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

clf = LogisticRegression(max_iter=1000, n_jobs=-1)  # add class_weight='balanced' if needed
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_valid_tfidf)
y_proba = clf.predict_proba(X_valid_tfidf)[:,1] if hasattr(clf, "predict_proba") else None

print("Accuracy:", accuracy_score(y_valid, y_pred))
if y_proba is not None and set(y_train.unique()) <= {0,1}:
    print("ROC-AUC:", roc_auc_score(y_valid, y_proba))
print(classification_report(y_valid, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred))

Train: (4771, 57912)  Valid: (1193, 57912)
Accuracy: 0.9790444258172674
ROC-AUC: 0.9970744163145661
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       677
           1       1.00      0.95      0.98       516

    accuracy                           0.98      1193
   macro avg       0.98      0.98      0.98      1193
weighted avg       0.98      0.98      0.98      1193

Confusion matrix:
 [[677   0]
 [ 25 491]]


In [35]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

svm = LinearSVC()  # you can tune C (default 1.0)
svm.fit(X_train_tfidf, y_train)

y_pred = svm.predict(X_valid_tfidf)
print("Accuracy:", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_valid, y_pred))

Accuracy: 0.9899413243922883
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       677
           1       1.00      0.98      0.99       516

    accuracy                           0.99      1193
   macro avg       0.99      0.99      0.99      1193
weighted avg       0.99      0.99      0.99      1193

Confusion matrix:
 [[677   0]
 [ 12 504]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code